In [ ]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import get_and_crawl_data
import gemini_model
import plotly_and_upload

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(get_and_crawl_data)
importlib.reload(gemini_model)
importlib.reload(plotly_and_upload)

from import_default import *
from import_database import *
from import_other import *
from get_and_crawl_data import *
from gemini_model import *
from plotly_and_upload import *

In [51]:
projection = {
    "_id": 0,
    "date": 1,
    "ticker": 1,
    "open": 1,
    "high": 1,
    "low": 1,
    "close": 1,
    "pct_change": 1,
    "diff": 1,
    "volume": 1,
    "option": 1,
    "SMA_20": 1,
    "SMA_60": 1,
    "RSI_14": 1,
    "week_open": 1,
    "month_prev_high": 1,
    "month_prev_low": 1,
    "month_open": 1,
    "quarter_prev_high": 1,
    "quarter_prev_low": 1,
    "quarter_open": 1,
    "year_open": 1,
    "MFIBO_0382": 1,
    "MFIBO_0500": 1,
    "MFIBO_0618": 1,
    "QFIBO_0382": 1,
    "QFIBO_0500": 1,
    "QFIBO_0618": 1,
	"YFIBO_0382": 1,
    "YFIBO_0500": 1,
    "YFIBO_0618": 1,
}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

In [ ]:
other_ticker_df = get_mongo_collection(stock_db, 'other_ticker', projection=projection)
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

In [ ]:
#Dữ liệu TD từ FiinProX
td_vn30f1m_df = pd.read_excel('../data/daily/ps_td.xlsx', skiprows=8).iloc[:,:12]
td_vn30f1m_df  = td_vn30f1m_df.dropna().iloc[:20].reset_index(drop=True)
td_vn30f1m_df = td_vn30f1m_df.drop(td_vn30f1m_df.columns[[3, 5, 7, 9]], axis=1)
td_vn30f1m_df.columns = ['ticker', 'date', 'buy_volume', 'sell_volume', 'buy_value', 'sell_value', 'net_value', 'net_volume']
td_vn30f1m_df[[col for col in td_vn30f1m_df.columns if '_value' in col]] = td_vn30f1m_df[[col for col in td_vn30f1m_df.columns if '_value' in col]]/1000
td_vn30f1m_df['type'] = 'TD'

#Lưu riêng ticker cho VN30F1M dùng sau này
vn30f1m_ticker = td_vn30f1m_df['ticker'].unique()[0]

#Dữ liệu NN từ Fireant
nn_vn30f1m_df = nntd_index_df[(nntd_index_df['ticker'] == 'VN30F1M') & (nntd_index_df['type'] == 'NN')].reset_index(drop=True)
nn_vn30f1m_df['ticker'] = vn30f1m_ticker

#Ghép bảng NN và TD
daily_8h45_nntd_df = pd.concat([nn_vn30f1m_df, td_vn30f1m_df], axis=0, ignore_index=True)

In [56]:
vn30f1m_chart_df = full_index_df[full_index_df['ticker'] == 'VNINDEX'].iloc[:120].sort_values('date').reset_index(drop=True)
vn30f1m_chart_df['ticker'] = vn30f1m_ticker
image_name = 'TA_DAILY_VN30F1M.png'

line_name_dict = {
    'SMA_20': 'SMA 20',
    'SMA_60': 'SMA 60',
    'week_open': 'WEEK OPEN',
    'month_open': 'MONTH OPEN',
    'quarter_open': 'QUARTER OPEN',
    'year_open': 'YEAR OPEN',
    'month_prev_high': 'LAST MHIGH',
    'month_prev_low': 'LAST MLOW',
    'quarter_prev_high': 'LAST QHIGH',
    'quarter_prev_low': 'LAST QLOW',
    'MFIBO_0382': 'MFIBO 0.382',
    'MFIBO_0500': 'MFIBO 0.500',
    'MFIBO_0618': 'MFIBO 0.618',
    'QFIBO_0382': 'QFIBO 0.382',
    'QFIBO_0500': 'QFIBO 0.500',
    'QFIBO_0618': 'QFIBO 0.618',
	'YFIBO_0382': 'YFIBO 0.382',
    'YFIBO_0500': 'YFIBO 0.500',
    'YFIBO_0618': 'YFIBO 0.618'
}

chart_config = create_chart_config(
	title_font_size=30,
	axis_font_size=24,
	tag_font_size=24,
	price_tag_font_size=24,
    min_spacing_ratio = 0.042,
	margin=dict(l=20, r=220, t=20, b=20, pad=10)
)

max_attempts = 5
for attempt in range(max_attempts):
    try:
        fig, fig_images = create_financial_chart(
            vn30f1m_chart_df, # DataFrame của bạn
            width=1400,
            height=1200,
            line_name_dict=line_name_dict,
            line_columns=list(line_name_dict.keys()),
            chart_config=chart_config,
            path='../output/cts_weekly_chart/',
            image_name = image_name
        )
        upload_to_r2(fig_images, image_name)
        break
    except Exception as e:
        if attempt == max_attempts - 1:
            raise e
# fig.show()